In [1]:
from bs4 import BeautifulSoup
import requests
import json

import pandas as pd
import time

In [2]:
# Define the cryptocurrencies you are analyzing 
cryptos = ['cardano', 'ethereum', 'polkadot', 'kusama', 'gnosis', 'status', 'cosmos', 'solana', 'celo',
           'ark', 'decentraland', 'chainlink', 'iota', 'blockstack', 'maker', 'havven', 'eos', 'bitcoin',
           'skale', 'lisk', 'ripple', 'litecoin', 'bitcoin-cash', 'stellar', 'bitcoin-cash-sv', 'monero', 'tron', 
           'nem', 'tezos', 'filecoin', 'leo-token', 'crypto-com-chain', 'neo', 'hedera-hashgraph', 'aptos', 'vega-protocol', 
           'internet-computer', 'avalanche-2', 'sui', 'optimism', 'near', 'audius']

In [3]:
len(cryptos)

42

In [4]:
# Column names that are useful

# name,symbol, categories, market cap, market cap ranking, current price, homepage, repos_url, 
# sentiment_votes_up_percentage, down_percentage, watchlist_profolio_users, coingecko_score, developer_score
# community_score, liquidity_score, public_interest_score, ath price, ath_date, atl price, atl date, price change % in 24 hrs, 7 days, 30 days, 90 days, 1 year
# total_supply, max_supply, ciculating supply, last_updated, community data: twitter followers, telegram_channel users, 
# developer data: forks, stars, pull requests mergered /contributors, commit_count_4_weeks

In [ ]:
# Define the URL for the "Get Coins Markets" endpoint with market cap sorting
url = 'https://api.coingecko.com/api/v3/coins/markets'
params = {
    'vs_currency': 'usd',  # Specify the currency for market cap (you can change this to other fiat or crypto)
    'order': 'market_cap_desc',  # Sort by market cap in descending order
    'per_page': 250,  # Number of cryptocurrencies to retrieve in each request
    'page': 1  # Start with the first page
}

# Initialize an empty list to store all the data
all_data = []

# Fetch data for all pages
while len(all_data) < 350:
    response = requests.get(url, params=params)

    if response.status_code == 200:
        # Parse the response as JSON and append the data to the list
        data = response.json()
        all_data.extend(data)
        params['page'] += 1  # Move to the next page for the next request
    else:
        print("Failed to retrieve data.")
        break

# Print the top 350 cryptocurrencies by market cap
if all_data:
    print("Top 350 Cryptocurrencies by Market Cap:")
    for index, crypto in enumerate(all_data[:350], start=1):
        name = crypto['name']
        market_cap = crypto['market_cap']
        print(f"{index}. {name}: ${market_cap:,}")
else:
    print("No data retrieved.")

In [ ]:
# Define the date you want to get the market cap for
date = '10-12-2020'  # December 10, 2020

# Initialize an empty dictionary to store the market cap data
market_cap_data = {}

# Make a request for each cryptocurrency
for crypto in cryptos:
    url = f'https://api.coingecko.com/api/v3/coins/{crypto}/history?date={date}'
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        market_cap = data['market_data']['market_cap']['usd']
        market_cap_data[crypto] = market_cap
    else:
        print(f"Failed to fetch data for {crypto}")

# Print the market cap data for each cryptocurrency
for crypto, cap in market_cap_data.items():
    print(f"{crypto} Market Cap on {date}: ${cap:,}")

In [53]:
# Define the API URL
url = 'https://api.coingecko.com/api/v3/coins/ripple'

# Make a GET request to the API
response = requests.get(url)

if response.status_code == 200:
    data = response.json()

    # Extract nested data
    developer_data = data.get("developer_data", {})
    community_data = data.get("community_data", {})
    market_data = data.get("market_data", {})
    links = data.get("links", {})

    # Create a dictionary with the desired information
    cardano_info = {
        "name": data.get("name", ""),
        "symbol": data.get("symbol", ""),
        "categories": ", ".join(data.get("categories", [])),
        "market_cap": market_data.get("market_cap", {}).get("usd", 0),
        "market_cap_ranking": data.get("market_cap_rank", 0),
        "current_price": market_data.get("current_price", {}).get("usd", 0),
        "homepage": links.get("homepage", [""])[0],
        "repos_url": links.get("repos_url", {}).get("github", [""])[0],
        "sentiment_votes_up_percentage": data.get("sentiment_votes_up_percentage", 0),
        "sentiment_votes_down_percentage": data.get("sentiment_votes_down_percentage", 0),
        "watchlist_portfolio_users": data.get("watchlist_portfolio_users", 0),
        "coingecko_rank": data.get('coingecko_rank', 0),
        "coingecko_score": data.get("coingecko_score", 0),
        "developer_score": data.get("developer_score", 0),
        "community_score": data.get("community_score", 0),
        "liquidity_score": data.get("liquidity_score", 0),
        "public_interest_score": data.get("public_interest_score", 0),
        "ath_price": market_data.get("ath", {}).get("usd", 0),
        "ath_date": market_data.get("ath_date", {}).get("usd", ""),
        "atl_price": market_data.get("atl", {}).get("usd", 0),
        "atl_date": market_data.get("atl_date", {}).get("usd", ""),
        "price_change_24h": market_data.get("price_change_percentage_24h", 0),
        "price_change_7d": market_data.get("price_change_percentage_7d", 0),
        "price_change_30d": market_data.get("price_change_percentage_30d", 0),
        "price_change_60d": market_data.get("price_change_percentage_60d", 0),
        "price_change_1y": market_data.get("price_change_percentage_1y", 0),
        "total_supply": market_data.get("total_supply", 0),
        "max_supply": market_data.get("max_supply", 0),
        "circulating_supply": market_data.get("circulating_supply", 0),
        "last_updated": data.get("last_updated", ""),
        "twitter_followers": community_data.get("twitter_followers", 0),
        "telegram_channel_users": community_data.get("telegram_channel_user_count", 0),
        "forks": developer_data.get("forks", 0),
        "stars": developer_data.get("stars", 0),
        "pull_requests_merged": developer_data.get("pull_requests_merged", 0),
        "contributors": developer_data.get("pull_request_contributors", 0),
        "commit_count_4_weeks": developer_data.get("commit_count_4_weeks", 0)
    }

    # Create a DataFrame from the dictionary
    df = pd.DataFrame([cardano_info])
df

,name,symbol,categories,market_cap,market_cap_ranking,current_price,homepage,repos_url,sentiment_votes_up_percentage,sentiment_votes_down_percentage,watchlist_portfolio_users,coingecko_rank,coingecko_score,developer_score,community_score,liquidity_score,public_interest_score,ath_price,ath_date,atl_price,atl_date,price_change_24h,price_change_7d,price_change_30d,price_change_60d,price_change_1y,total_supply,max_supply,circulating_supply,last_updated,twitter_followers,telegram_channel_users,forks,stars,pull_requests_merged,contributors,commit_count_4_weeks
0,XRP,xrp,"FTX Holdings, Cryptocurrency",35280429772,5,0.656811,https://ripple.com/currency/,https://github.com/ripple/rippled,76.28,23.72,583993,3,66.223,83.005,51.794,81.07,0.022,3.4,2018-01-07T00:00:00.000Z,0.002686,2014-05-22T00:00:00.000Z,0.0905,7.83305,36.32795,39.35402,67.55468,9.998824e+10,1.000000e+11,5.365277e+10,2023-11-10T17:05:30.066Z,2696601,None,1512,4361,618,60,21


In [15]:
df.columns

Index(['name', 'symbol', 'categories', 'market_cap', 'market_cap_ranking',
       'current_price', 'homepage', 'repos_url',
       'sentiment_votes_up_percentage', 'sentiment_votes_down_percentage',
       'watchlist_portfolio_users', 'coingecko_score', 'developer_score',
       'community_score', 'liquidity_score', 'public_interest_score',
       'ath_price', 'ath_date', 'atl_price', 'atl_date', 'price_change_24h',
       'price_change_7d', 'price_change_30d', 'price_change_90d',
       'price_change_1y', 'total_supply', 'max_supply', 'circulating_supply',
       'last_updated', 'twitter_followers', 'telegram_channel_users', 'forks',
       'stars', 'pull_requests_merged', 'contributors',
       'commit_count_4_weeks'],
      dtype='object')

In [20]:
developer_data = data.get("developer_data", {})
developer_data

{'forks': 641,
 'stars': 3756,
 'subscribers': 466,
 'total_issues': 849,
 'closed_issues': 665,
 'pull_requests_merged': 1706,
 'pull_request_contributors': 80,
 'code_additions_deletions_4_weeks': {'additions': 0, 'deletions': 0},
 'commit_count_4_weeks': 0,
 'last_4_weeks_commit_activity_series': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [54]:
pd.options.display.max_columns = None
df

,name,symbol,categories,market_cap,market_cap_ranking,current_price,homepage,repos_url,sentiment_votes_up_percentage,sentiment_votes_down_percentage,watchlist_portfolio_users,coingecko_rank,coingecko_score,developer_score,community_score,liquidity_score,public_interest_score,ath_price,ath_date,atl_price,atl_date,price_change_24h,price_change_7d,price_change_30d,price_change_60d,price_change_1y,total_supply,max_supply,circulating_supply,last_updated,twitter_followers,telegram_channel_users,forks,stars,pull_requests_merged,contributors,commit_count_4_weeks
0,XRP,xrp,"FTX Holdings, Cryptocurrency",35280429772,5,0.656811,https://ripple.com/currency/,https://github.com/ripple/rippled,76.28,23.72,583993,3,66.223,83.005,51.794,81.07,0.022,3.4,2018-01-07T00:00:00.000Z,0.002686,2014-05-22T00:00:00.000Z,0.0905,7.83305,36.32795,39.35402,67.55468,9.998824e+10,1.000000e+11,5.365277e+10,2023-11-10T17:05:30.066Z,2696601,None,1512,4361,618,60,21


In [51]:
data

{'id': 'ripple',
 'symbol': 'xrp',
 'name': 'XRP',
 'asset_platform_id': None,
 'platforms': {'': ''},
 'detail_platforms': {'': {'decimal_place': None, 'contract_address': ''}},
 'block_time_in_minutes': 0,
 'hashing_algorithm': None,
 'categories': ['FTX Holdings', 'Cryptocurrency'],
 'preview_listing': False,
 'public_notice': None,
 'additional_notices': [],
 'localization': {'en': 'XRP',
  'de': 'XRP',
  'es': 'XRP',
  'fr': 'XRP',
  'it': 'XRP',
  'pl': 'XRP',
  'ro': 'XRP',
  'hu': 'XRP',
  'nl': 'XRP',
  'pt': 'XRP',
  'sv': 'XRP',
  'vi': 'XRP',
  'tr': 'XRP',
  'ru': 'Ripple',
  'ja': 'リップル',
  'zh': '瑞波币',
  'zh-tw': '瑞波幣',
  'ko': '리플코인',
  'ar': 'XRP',
  'th': 'XRP',
  'id': 'XRP',
  'cs': 'XRP',
  'da': 'XRP',
  'el': 'XRP',
  'hi': 'XRP',
  'no': 'XRP',
  'sk': 'XRP',
  'uk': 'XRP',
  'he': 'XRP',
  'fi': 'XRP',
  'bg': 'XRP',
  'hr': 'XRP',
  'lt': 'XRP',
  'sl': 'XRP'},
 'description': {'en': 'Ripple is the catchall name for the cryptocurrency platform, the transaction

### Notes:

##### Data from 12/10/20 & 3/13/20

- prices / mkt cap / mkt cap ranking
- developer info, forks	stars, pull_requests_merged, pull_requests_contributors, commit_count_4_weeks
- twitter_followers, telegram_channel_users	


*sentiment_votes_up_percentage, sentiment_votes_down_percentage, watchlist_portfolio_users, <br>
coingecko_rank, coingecko_score, developer_score, community_score, liquidity_score, public_interest_score, <br>
total_supply, max_supply, circulating_supply*


In [60]:
# Define the date you want to get the market cap for
date = '10-12-2020'  # December 10, 2020
url = f'https://api.coingecko.com/api/v3/coins/cardano/history?date={date}'
response = requests.get(url)
data = response.json()
 

In [67]:
#data['market_data']['current_price']['usd']
data.keys()

dict_keys(['id', 'symbol', 'name', 'localization', 'image', 'market_data', 'community_data', 'developer_data', 'public_interest_stats'])

In [105]:
mkt_data = data.get('market_data',0)
com_data = data.get('community_data', 0)
dev_data = data.get('developer_data', 0)

historical_data = {
    'name': data.get('name',{}),
    'dev_price': mkt_data.get('current_price', 0).get('usd', 0),
    'dev_mkt_cap': mkt_data.get('market_cap', 0).get('usd', 0),
    'dev_twitter_followers': com_data.get('twitter_followers', 0),
    'dev_reddit_post_48h': com_data.get('reddit_average_posts_48h', 0),
    'dev_reddit_avg_comments_48h': com_data.get('reddit_average_comments_48h', 0),
    'dev_reddit_subs': com_data.get('reddit_subscribers', 0),
    'dev_reddit_accounts_active_48h': com_data.get('reddit_accounts_active_48h', 0),
    'dev_forks': dev_data.get('forks', 0),
    'dev_stars':dev_data.get('stars', 0), 
    'dev_pull_merged': dev_data.get('pull_requests_merged', 0),
    'dev_pull_contributors': dev_data.get('pull_request_contributors', 0),
    }
historical_data

df = pd.DataFrame([historical_data])
df

,name,dev_price,dev_mkt_cap,dev_twitter_followers,dev_reddit_post_48h,dev_reddit_avg_comments_48h,dev_reddit_subs,dev_reddit_accounts_active_48h,dev_forks,dev_stars,dev_pull_merged,dev_pull_contributors
0,Cardano,0.148249,4.620146e+09,167733,3.636,40.818,91917,782.666666666667,578,3425,1706,80


- developer report price = 12/10/2020 (dev_***)
- covid-19 prices bottom = 03/12/2020 (c19_***)

In [103]:
data['developer_data']

{'forks': 578,
 'stars': 3425,
 'subscribers': 409,
 'total_issues': 850,
 'closed_issues': 665,
 'pull_requests_merged': 1706,
 'pull_request_contributors': 80,
 'code_additions_deletions_4_weeks': {'additions': 0, 'deletions': 0},
 'commit_count_4_weeks': 0}

In [108]:
#covid19 prices
date = '13-03-2020' 
url = f'https://api.coingecko.com/api/v3/coins/cardano/history?date={date}'
response = requests.get(url)
data = response.json()

In [109]:
mkt_data = data.get('market_data',0)
com_data = data.get('community_data', 0)
dev_data = data.get('developer_data', 0)

c19_data = {
    'name': data.get('name',{}),
    'c19_price': mkt_data.get('current_price', 0).get('usd', 0),
    'c19_mkt_cap': mkt_data.get('market_cap', 0).get('usd', 0),
    'c19_twitter_followers': com_data.get('twitter_followers', 0),
    'c19_reddit_post_48h': com_data.get('reddit_average_posts_48h', 0),
    'c19_reddit_avg_comments': com_data.get('reddit_average_comments_48h', 0),
    'c19_reddit_subs': com_data.get('reddit_subscribers', 0),
    'c19_reddit_accounts_active': com_data.get('reddit_accounts_active_48h', 0),
    'c19_forks': dev_data.get('forks', 0),
    'c19_stars':dev_data.get('stars', 0), 
    'c19_pull_merged': dev_data.get('pull_requests_merged', 0),
    'c19_pull_contributors': dev_data.get('pull_request_contributors', 0),
    }
c19_data
df = pd.DataFrame([c19_data])
df

,name,c19_price,c19_mkt_cap,c19_twitter_followers,c19_reddit_post_48h,c19_reddit_avg_comments,c19_reddit_subs,c19_reddit_accounts_active,c19_forks,c19_stars,c19_pull_merged,c19_pull_contributors
0,Cardano,0.023716,7.335692e+08,155706,2.0,21.0,77107,816.916666666667,583,3384,1703,80


In [118]:
dates = {'d1':'12-03-2020', 'd2':'13-03-2020'}
comparing_prices = {}

for key, date in dates.items():
    url = f'https://api.coingecko.com/api/v3/coins/cardano/history?date={date}'
    response = requests.get(url)
    data = response.json()
    
    price = data.get('market_data').get('current_price').get('usd')
    
    comparing_prices[key] = price
    
        
comparing_prices    

{'d1': 0.040066910703005464, 'd2': 0.023715965392713114}

In [141]:
# Given list of cryptocurrencies
#cryptos = ['cardano', 'bitcoin', 'ethereum', 'ripple', 'litecoin', 'chainlink', 'polkadot', 'stellar', 'tezos', 'eos']

# Dictionary to store data for each crypto
crypto_data_list = []

# Iterate over each crypto
for crypto in cryptos:
    # Dictionary to store prices for each date
    comparing_prices = {}

    # Given dates
    dates = {'d1': '12-03-2020', 'd2': '13-03-2020'}

    # Fetch prices for each date
    for key, date in dates.items():
        url = f'https://api.coingecko.com/api/v3/coins/{crypto}/history?date={date}'
        response = requests.get(url)
        data = response.json()

        # Extract the price
        price = data.get('market_data', {}).get('current_price', {}).get('usd')

        # Store the price in the dictionary
        comparing_prices[key] = price
    time.sleep(10)

           
    crypto_data_list.append({crypto: comparing_prices})

crypto_data_list
    
    # Find the date with the lowest price
    #lowest_price_date = min(comparing_prices, key=comparing_prices.get)

    # Continue with data collection using the date with the lowest price
    #selected_date = dates[lowest_price_date]


[{'cardano': {'d1': 0.040066910703005464, 'd2': 0.023715965392713114}},
 {'ethereum': {'d1': 194.21794124025206, 'd2': 110.5978978308351}},
 {'polkadot': {'d1': None, 'd2': None}},
 {'kusama': {'d1': 2.0171066654337966, 'd2': 1.4201580123892796}},
 {'gnosis': {'d1': 14.210803663686402, 'd2': 9.348050813764349}},
 {'status': {'d1': 0.012929373016390229, 'd2': 0.00808531218668294}},
 {'cosmos': {'d1': 2.9935198148139266, 'd2': 1.6092393792935413}},
 {'solana': {'d1': None, 'd2': None}},
 {'celo': {'d1': None, 'd2': None}},
 {'ark': {'d1': 0.1875974898416054, 'd2': 0.09118947339286429}},
 {'decentraland': {'d1': 0.03505547669868683, 'd2': 0.01823543746517608}},
 {'chainlink': {'d1': 3.8232672229681817, 'd2': 1.9744280382744082}},
 {'iota': {'d1': 0.19527694503790835, 'd2': 0.11140009590221976}},
 {'blockstacks': {'d1': None, 'd2': None}},
 {'maker': {'d1': 490.6889438293519, 'd2': 203.3193186247471}},
 {'synthetix': {'d1': None, 'd2': None}},
 {'eos': {'d1': 3.0739085323634905, 'd2': 1.88

In [133]:
comparing_prices

{'d1': None, 'd2': None}

Acquiring stage:

- figured out how call the api to reach correct endpoints
- got 42 cryptos that we know are either part of the top 20 MC 2020/2023 and top 20 developer report 2020/2023
- selected the features that we thought would be useful according to the data provided within current market
- selected more features from the historical api that were thought to be useful according to dec. 10th, 2020 which is when the developer report came out before bull run of 2020/2021
- decided to get same features, but from a different date which was the low of covid-19; had to compared dates to see which was overall the bottom of the market at that time between 03/12/2020 & 03/13/2020
- March 13th, 2020 came back as the best date 
- Put calling of api's all into a function to run both historical data and current data

In [16]:
# Create a DataFrame from the dictionary
crypto_data = []

for crypto in cryptos:
    # Define the API URL
    url = f'https://api.coingecko.com/api/v3/coins/{crypto}'

    # Make a GET request to the API
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        # Extract nested data
        developer_data = data.get("developer_data", {})
        community_data = data.get("community_data", {})
        market_data = data.get("market_data", {})
        links = data.get("links", {})

        # Create a dictionary with the desired information
        current_crypto_data = {
            "name": data.get("name", ""),
            "symbol": data.get("symbol", ""),
            "categories": ", ".join(data.get("categories", [])),
            "market_cap": market_data.get("market_cap", {}).get("usd", 0),
            "market_cap_ranking": data.get("market_cap_rank", 0),
            "current_price": market_data.get("current_price", {}).get("usd", 0),
            "homepage": links.get("homepage", [""])[0],
            "repos_url": links.get("repos_url", {}).get("github", ""),
            "sentiment_votes_up_percentage": data.get("sentiment_votes_up_percentage", 0),
            "sentiment_votes_down_percentage": data.get("sentiment_votes_down_percentage", 0),
            "watchlist_portfolio_users": data.get("watchlist_portfolio_users", 0),
            "coingecko_rank": data.get('coingecko_rank', 0),
            "coingecko_score": data.get("coingecko_score", 0),
            "developer_score": data.get("developer_score", 0),
            "community_score": data.get("community_score", 0),
            "liquidity_score": data.get("liquidity_score", 0),
            "public_interest_score": data.get("public_interest_score", 0),
            "ath_price": market_data.get("ath", {}).get("usd", 0),
            "ath_date": market_data.get("ath_date", {}).get("usd", ""),
            "atl_price": market_data.get("atl", {}).get("usd", 0),
            "atl_date": market_data.get("atl_date", {}).get("usd", ""),
            "price_change_24h": market_data.get("price_change_percentage_24h", 0),
            "price_change_7d": market_data.get("price_change_percentage_7d", 0),
            "price_change_30d": market_data.get("price_change_percentage_30d", 0),
            "price_change_60d": market_data.get("price_change_percentage_60d", 0),
            "price_change_1y": market_data.get("price_change_percentage_1y", 0),
            "total_supply": market_data.get("total_supply", 0),
            "max_supply": market_data.get("max_supply", 0),
            "circulating_supply": market_data.get("circulating_supply", 0),
            "last_updated": data.get("last_updated", ""),
            "twitter_followers": community_data.get("twitter_followers", 0),
            "telegram_channel_users": community_data.get("telegram_channel_user_count", 0),
            "forks": developer_data.get("forks", 0),
            "stars": developer_data.get("stars", 0),
            "pull_requests_merged": developer_data.get("pull_requests_merged", 0),
            "contributors": developer_data.get("pull_request_contributors", 0),
            "commit_count_4_weeks": developer_data.get("commit_count_4_weeks", 0)
        }
        
        
        crypto_data.append(current_crypto_data)
        time.sleep(5)

In [15]:
len(cryptos)

42

In [17]:
df = pd.DataFrame(crypto_data)

In [18]:
df

,name,symbol,categories,market_cap,market_cap_ranking,current_price,homepage,repos_url,sentiment_votes_up_percentage,sentiment_votes_down_percentage,...,max_supply,circulating_supply,last_updated,twitter_followers,telegram_channel_users,forks,stars,pull_requests_merged,contributors,commit_count_4_weeks
0,Cardano,ada,"Alleged SEC Securities, Smart Contract Platfor...",13300688622,9,0.380214,https://www.cardano.org/en/home/,[https://github.com/input-output-hk/cardano-sl...,90.75,9.25,...,4.500000e+10,3.495476e+10,2023-11-13T03:55:06.739Z,834370,23145.0,641,3756,1706,80,0
1,Ethereum,eth,"FTX Holdings, Smart Contract Platform, Layer 1...",246481042382,2,2045.940000,https://www.ethereum.org/,"[https://github.com/ethereum/go-ethereum, http...",84.58,15.42,...,NaN,1.202603e+08,2023-11-13T03:55:21.133Z,3162243,NaN,18870,44022,5678,881,60
2,Polkadot,dot,"Smart Contract Platform, Polkadot Ecosystem, L...",7411031602,14,5.700000,https://polkadot.network/,[https://github.com/paritytech/polkadot],92.16,7.84,...,NaN,1.296001e+09,2023-11-13T03:55:11.993Z,1422820,24992.0,1629,7051,4126,253,0
3,Kusama,ksm,Polkadot Ecosystem,239315522,167,26.470000,https://kusama.network/,[https://github.com/kusamanetwork/kusama.network],95.45,4.55,...,1.000000e+07,8.980098e+06,2023-11-13T03:55:56.183Z,254723,NaN,0,0,0,0,0
4,Gnosis,gno,"Decentralized Exchange (DEX), Protocol, Smart ...",397662770,122,153.380000,https://gnosis.io/,"[https://github.com/gnosis/safe-contracts/, ht...",66.67,33.33,...,3.000000e+06,2.589589e+06,2023-11-13T03:56:00.100Z,119052,NaN,22,66,111,9,0
5,Status,snt,"Software, Ethereum Ecosystem",179817913,207,0.045393,https://status.im/,"[https://github.com/status-im/status-react, ht...",100.00,0.00,...,NaN,3.960484e+09,2023-11-13T03:55:44.573Z,150074,NaN,998,3740,6485,241,118
6,Cosmos Hub,atom,"Alleged SEC Securities, Cryptocurrency, BNB Ch...",2791953738,29,9.510000,http://cosmos.network/,"[https://github.com/cosmos/cosmos, https://git...",90.14,9.86,...,NaN,2.925862e+08,2023-11-13T03:55:55.671Z,520334,16875.0,320,1218,39,21,0
7,Solana,sol,"Alleged SEC Securities, FTX Holdings, Smart Co...",24312342572,6,57.440000,https://solana.com/,[https://github.com/solana-labs/solana],81.47,18.53,...,NaN,4.221652e+08,2023-11-13T03:55:28.612Z,2279605,52112.0,3311,10769,23182,403,195
8,Celo,celo,"DWF Labs Portfolio, Celo Ecosystem",294719834,150,0.567598,https://celo.org/,"[https://github.com/celo-org/celo-blockchain, ...",80.00,20.00,...,1.000000e+09,5.183573e+08,2023-11-13T03:55:39.952Z,451197,8394.0,190,502,1046,65,9
9,ARK,ark,Cryptocurrency,302852356,145,1.690000,https://ark.io/,"[https://github.com/ArkEcosystem/core, https:/...",75.00,25.00,...,NaN,1.768764e+08,2023-11-13T03:56:00.456Z,76173,1766.0,301,333,3389,48,0


In [19]:
df.shape

(42, 37)

In [215]:
df['name'].str.lower().tolist()

['cardano',
 'ethereum',
 'polkadot',
 'kusama',
 'gnosis',
 'status',
 'cosmos hub',
 'solana',
 'celo',
 'ark',
 'decentraland',
 'chainlink',
 'iota',
 'maker',
 'eos',
 'bitcoin',
 'skale',
 'lisk',
 'xrp',
 'litecoin',
 'stellar',
 'monero',
 'tron',
 'nem',
 'tezos',
 'filecoin',
 'neo',
 'aptos',
 'sui',
 'optimism',
 'audius']

In [195]:
cryptos

['cardano',
 'ethereum',
 'polkadot',
 'kusama',
 'gnosis',
 'status',
 'cosmos',
 'solana',
 'celo',
 'ark',
 'decentraland',
 'chainlink',
 'iota',
 'blockstacks',
 'maker',
 'synthetix',
 'eos',
 'bitcoin',
 'skale',
 'lisk',
 'ripple',
 'litecoin',
 'bitcoin cash',
 'stellar',
 'bitcoin sv',
 'monero',
 'tron',
 'nem',
 'tezos',
 'filecoin',
 'unus sed leo',
 'cronos',
 'neo',
 'hedera',
 'aptos',
 'vega protocol',
 'internet computer',
 'avalanche',
 'sui',
 'optimism',
 'near protocol',
 'audius']

In [228]:
not_in_df= []

for crypto in cryptos:
    
    if crypto not in df['name'].str.lower().tolist() and crypto not in ['ripple','cosmos']:
        
        not_in_df.append(crypto)

not_in_df

['blockstacks',
 'synthetix',
 'bitcoin cash',
 'bitcoin sv',
 'unus sed leo',
 'cronos',
 'hedera',
 'vega protocol',
 'internet computer',
 'avalanche',
 'near protocol']

In [5]:
url = 'https://api.coingecko.com/api/v3/coins/stacks'
response = requests.get(url)
data = response.json()
data['links']

{'homepage': ['https://stackbnb.com/', '', ''],
 'blockchain_site': ['https://bscscan.com/address/0x14220f5893e945eed36389905b991d6b972e04f0',
  'https://bscscan.com/token/0x14220f5893e945eed36389905b991d6b972e04f0',
  'https://binplorer.com/address/0x14220f5893e945eed36389905b991d6b972e04f0',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 'official_forum_url': ['https://www.tiktok.com/@stackstokenbsc', '', ''],
 'chat_url': ['', 'https://medium.com/@stackstokenbsc', ''],
 'announcement_url': ['', ''],
 'twitter_screen_name': 'StacksTokenBSC',
 'facebook_username': '',
 'bitcointalk_thread_identifier': None,
 'telegram_channel_identifier': 'StacksTokenBSC',
 'subreddit_url': 'https://www.reddit.com',
 'repos_url': {'github': [], 'bitbucket': []}}

In [17]:
prices = {}    
    
for crypto in cryptos:
    
    url = f'https://api.coingecko.com/api/v3/coins/{crypto}'

    # Make a GET request to the API
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        
        name = data.get('name')
        price = data.get('market_data').get('current_price').get('usd')    
        
        prices[name] = price
        time.sleep(5)
        
pd.DataFrame(prices.items(), columns=['Crypto', 'Price (USD)'])        
        

,Crypto,Price (USD)
0,Cardano,0.363245
1,Ethereum,2040.980000
2,Polkadot,5.320000
3,Kusama,24.960000
4,Gnosis,174.700000
5,Status,0.044779
6,Cosmos Hub,9.310000
7,Solana,54.380000
8,Celo,0.546646
9,ARK,1.570000


In [33]:
pd.DataFrame(prices.items(),columns=['Crypto', 'USD']).set_index('Crypto')

,USD
Crypto,
Cardano,0.363245
Ethereum,2040.980000
Polkadot,5.320000
Kusama,24.960000
Gnosis,174.700000
Status,0.044779
Cosmos Hub,9.310000
Solana,54.380000
Celo,0.546646


In [32]:
list(prices.items())

[('Cardano', 0.363245),
 ('Ethereum', 2040.98),
 ('Polkadot', 5.32),
 ('Kusama', 24.96),
 ('Gnosis', 174.7),
 ('Status', 0.04477897),
 ('Cosmos Hub', 9.31),
 ('Solana', 54.38),
 ('Celo', 0.546646),
 ('ARK', 1.57),
 ('Decentraland', 0.432863),
 ('Chainlink', 14.27),
 ('IOTA', 0.188247)]

In [54]:
c19_data = []

for crypto in cryptos:
    
    date = '13-03-2020' 
    
    # Define the API URL
    url = f'https://api.coingecko.com/api/v3/coins/{crypto}/history?date={date}'

    # Make a GET request to the API
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        mkt_data = data.get('market_data',{})
        com_data = data.get('community_data', {})
        dev_data = data.get('developer_data', {})

        c19 = {
            'name': data.get('name',{}),
            'c19_price': mkt_data.get('current_price', {}).get('usd', 0),
            'c19_mkt_cap': mkt_data.get('market_cap', {}).get('usd', 0),
            'c19_twitter_followers': com_data.get('twitter_followers', 0),
            'c19_reddit_post_48h': com_data.get('reddit_average_posts_48h', 0),
            'c19_reddit_avg_comments': com_data.get('reddit_average_comments_48h', 0),
            'c19_reddit_subs': com_data.get('reddit_subscribers', 0),
            'c19_reddit_accounts_active': com_data.get('reddit_accounts_active_48h', 0),
            'c19_forks': dev_data.get('forks', 0),
            'c19_stars':dev_data.get('stars', 0), 
            'c19_pull_merged': dev_data.get('pull_requests_merged', 0),
            'c19_pull_contributors': dev_data.get('pull_request_contributors', 0),
            }
        
        c19_data.append(c19)
        time.sleep(7)            
        


In [55]:
len(c19_data)

42

In [56]:
df = pd.DataFrame(c19_data)
df

,name,c19_price,c19_mkt_cap,c19_twitter_followers,c19_reddit_post_48h,c19_reddit_avg_comments,c19_reddit_subs,c19_reddit_accounts_active,c19_forks,c19_stars,c19_pull_merged,c19_pull_contributors
0,Cardano,0.023716,7.335692e+08,155706.0,2.000,21.000,77107.0,816.916666666667,583.0,3384.0,1703.0,80.0
1,Ethereum,110.597898,1.195663e+10,452873.0,3.455,43.091,454613.0,1439.5,9286.0,25527.0,2869.0,419.0
2,Polkadot,0.000000,0.000000e+00,0.0,0.000,0.000,0.0,0,0.0,0.0,0.0,0.0
3,Kusama,1.420158,9.209000e+06,4746.0,0.000,0.000,NaN,None,NaN,NaN,NaN,NaN
4,Gnosis,9.348051,1.022584e+07,71361.0,0.091,0.091,2153.0,42.8333333333333,19.0,62.0,111.0,9.0
5,Status,0.008085,2.942284e+07,110605.0,0.000,0.000,5609.0,41.1666666666667,735.0,2870.0,3528.0,173.0
6,Cosmos Hub,1.609239,2.957098e+08,31661.0,0.364,0.818,8256.0,95.5,168.0,846.0,39.0,21.0
7,Solana,0.000000,0.000000e+00,0.0,0.000,0.000,0.0,0,0.0,0.0,0.0,0.0
8,Celo,0.000000,0.000000e+00,0.0,0.000,0.000,0.0,0,0.0,0.0,0.0,0.0
9,ARK,0.091189,1.313253e+07,62703.0,0.545,4.545,21959.0,396.916666666667,172.0,256.0,2491.0,44.0


In [60]:
df[df['c19_price']  ==  0]

,name,c19_price,c19_mkt_cap,c19_twitter_followers,c19_reddit_post_48h,c19_reddit_avg_comments,c19_reddit_subs,c19_reddit_accounts_active,c19_forks,c19_stars,c19_pull_merged,c19_pull_contributors
2,Polkadot,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
7,Solana,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
8,Celo,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
18,SKALE,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
29,Filecoin,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
34,Aptos,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
35,Vega Protocol,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
36,Internet Computer,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
37,Avalanche,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
38,Sui,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0


In [63]:
df.shape

(42, 12)

In [66]:
df.to_csv('c19_data', index = False)

In [12]:
all_dev_data = []

for crypto in cryptos:
    
    date = '10-12-2020' 
    
    # Define the API URL
    url = f'https://api.coingecko.com/api/v3/coins/{crypto}/history?date={date}'

    # Make a GET request to the API
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        
        mkt_data = data.get('market_data', {})
        com_data = data.get('community_data', {})
        dev_data = data.get('developer_data', {})

        historical_data = {
            'name': data.get('name',{}),
            'dev_price': mkt_data.get('current_price', {}).get('usd', 0),
            'dev_mkt_cap': mkt_data.get('market_cap', {}).get('usd', 0),
            'dev_twitter_followers': com_data.get('twitter_followers', 0),
            'dev_reddit_post_48h': com_data.get('reddit_average_posts_48h', 0),
            'dev_reddit_avg_comments_48h': com_data.get('reddit_average_comments_48h', 0),
            'dev_reddit_subs': com_data.get('reddit_subscribers', 0),
            'dev_reddit_accounts_active_48h': com_data.get('reddit_accounts_active_48h', 0),
            'dev_forks': dev_data.get('forks', 0),
            'dev_stars':dev_data.get('stars', 0), 
            'dev_pull_merged': dev_data.get('pull_requests_merged', 0),
            'dev_pull_contributors': dev_data.get('pull_request_contributors', 0),
            }

        all_dev_data.append(historical_data)
        time.sleep(7)        

In [13]:
len(all_dev_data)

42

In [14]:
all_dev_data

[{'name': 'Cardano',
  'dev_price': 0.14824925464106553,
  'dev_mkt_cap': 4620145846.073955,
  'dev_twitter_followers': 167733,
  'dev_reddit_post_48h': 3.636,
  'dev_reddit_avg_comments_48h': 40.818,
  'dev_reddit_subs': 91917,
  'dev_reddit_accounts_active_48h': '782.666666666667',
  'dev_forks': 578,
  'dev_stars': 3425,
  'dev_pull_merged': 1706,
  'dev_pull_contributors': 80},
 {'name': 'Ethereum',
  'dev_price': 573.8953091099941,
  'dev_mkt_cap': 65276424697.152855,
  'dev_twitter_followers': 521978,
  'dev_reddit_post_48h': 2.909,
  'dev_reddit_avg_comments_48h': 63.909,
  'dev_reddit_subs': 501708,
  'dev_reddit_accounts_active_48h': '1249.0',
  'dev_forks': 10084,
  'dev_stars': 27382,
  'dev_pull_merged': 3365,
  'dev_pull_contributors': 485},
 {'name': 'Polkadot',
  'dev_price': 4.863467872055046,
  'dev_mkt_cap': 4580216020.823783,
  'dev_twitter_followers': 79273,
  'dev_reddit_post_48h': 0.455,
  'dev_reddit_avg_comments_48h': 3.545,
  'dev_reddit_subs': 6320,
  'dev_red

In [15]:
all_dev_df = pd.DataFrame(all_dev_data)
all_dev_df

,name,dev_price,dev_mkt_cap,dev_twitter_followers,dev_reddit_post_48h,dev_reddit_avg_comments_48h,dev_reddit_subs,dev_reddit_accounts_active_48h,dev_forks,dev_stars,dev_pull_merged,dev_pull_contributors
0,Cardano,0.148249,4.620146e+09,167733.0,3.636,40.818,91917.0,782.666666666667,578.0,3425.0,1706.0,80.0
1,Ethereum,573.895309,6.527642e+10,521978.0,2.909,63.909,501708.0,1249.0,10084.0,27382.0,3365.0,485.0
2,Polkadot,4.863468,4.580216e+09,79273.0,0.455,3.545,6320.0,36.5833333333333,NaN,NaN,NaN,NaN
3,Kusama,49.074908,4.405368e+08,16207.0,0.000,0.000,NaN,None,NaN,NaN,NaN,NaN
4,Gnosis,61.448445,6.788059e+07,72017.0,0.000,0.000,2217.0,22.9166666666667,19.0,61.0,111.0,9.0
5,Status,0.034372,1.335002e+08,113893.0,0.000,0.000,5685.0,1342.16666666667,771.0,3014.0,4164.0,186.0
6,Cosmos Hub,4.799491,1.144911e+09,45748.0,0.417,1.000,9810.0,33.5384615384615,184.0,896.0,39.0,21.0
7,Solana,1.664937,7.725853e+07,102350.0,1.000,5.455,2692.0,26.4166666666667,250.0,1070.0,10074.0,80.0
8,Celo,1.674516,0.000000e+00,10202.0,0.000,0.000,NaN,None,56.0,174.0,591.0,46.0
9,ARK,0.386712,5.928320e+07,62874.0,0.182,3.364,21644.0,94.3333333333333,275.0,279.0,2924.0,46.0


In [16]:
all_dev_df.to_csv('develop_data', index = False)

In [92]:
date = '10-12-2020' 
    
    # Define the API URL
url = f'https://api.coingecko.com/api/v3/coins/ark/history?date={date}'

    # Make a GET request to the API
response = requests.get(url)
data= response.json()
data

{'id': 'ark',
 'symbol': 'ark',
 'name': 'ARK',
 'localization': {'en': 'ARK',
  'de': 'ARK',
  'es': 'ARK',
  'fr': 'ARK',
  'it': 'ARK',
  'pl': 'ARK',
  'ro': 'ARK',
  'hu': 'ARK',
  'nl': 'ARK',
  'pt': 'ARK',
  'sv': 'ARK',
  'vi': 'ARK',
  'tr': 'ARK',
  'ru': 'ARK',
  'ja': 'アーク',
  'zh': 'ARK',
  'zh-tw': 'ARK',
  'ko': '아크',
  'ar': 'ARK',
  'th': 'ARK',
  'id': 'ARK',
  'cs': 'ARK',
  'da': 'ARK',
  'el': 'ARK',
  'hi': 'ARK',
  'no': 'ARK',
  'sk': 'ARK',
  'uk': 'ARK',
  'he': 'ARK',
  'fi': 'ARK',
  'bg': 'ARK',
  'hr': 'ARK',
  'lt': 'ARK',
  'sl': 'ARK'},
 'image': {'thumb': 'https://assets.coingecko.com/coins/images/613/thumb/ark.png?1696501811',
  'small': 'https://assets.coingecko.com/coins/images/613/small/ark.png?1696501811'},
 'market_data': {'current_price': {'aed': 1.420471826613463,
   'ars': 31.701402806475073,
   'aud': 0.5202050747818259,
   'bch': 0.0014375457426405035,
   'bdt': 32.78382925353453,
   'bhd': 0.14577586319935326,
   'bmd': 0.38671235615089394

In [93]:
data.keys()

dict_keys(['id', 'symbol', 'name', 'localization', 'image', 'market_data', 'community_data', 'developer_data', 'public_interest_stats'])

In [17]:
def developer_data_date(cryptos, file_name):
    
    all_dev_data = []

    for crypto in cryptos:

        date = '10-12-2020' 

        # Define the API URL
        url = f'https://api.coingecko.com/api/v3/coins/{crypto}/history?date={date}'

        # Make a GET request to the API
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()

            mkt_data = data.get('market_data', {})
            com_data = data.get('community_data', {})
            dev_data = data.get('developer_data', {})

            historical_data = {
                'name': data.get('name',{}),
                'dev_price': mkt_data.get('current_price', {}).get('usd', 0),
                'dev_mkt_cap': mkt_data.get('market_cap', {}).get('usd', 0),
                'dev_twitter_followers': com_data.get('twitter_followers', 0),
                'dev_reddit_post_48h': com_data.get('reddit_average_posts_48h', 0),
                'dev_reddit_avg_comments_48h': com_data.get('reddit_average_comments_48h', 0),
                'dev_reddit_subs': com_data.get('reddit_subscribers', 0),
                'dev_reddit_accounts_active_48h': com_data.get('reddit_accounts_active_48h', 0),
                'dev_forks': dev_data.get('forks', 0),
                'dev_stars':dev_data.get('stars', 0), 
                'dev_pull_merged': dev_data.get('pull_requests_merged', 0),
                'dev_pull_contributors': dev_data.get('pull_request_contributors', 0),
                }

            all_dev_data.append(historical_data)
            time.sleep(7)  
            
    all_dev_df = pd.DataFrame(all_dev_data)
    all_dev_df.to_csv(file_name, index = False)
    
    return all_dev_df

In [18]:
def c19_data_date(cryptos, file_name):
    
    c19_data = []

    for crypto in cryptos:

        date = '13-03-2020' 

        # Define the API URL
        url = f'https://api.coingecko.com/api/v3/coins/{crypto}/history?date={date}'

        # Make a GET request to the API
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()

            mkt_data = data.get('market_data',{})
            com_data = data.get('community_data', {})
            dev_data = data.get('developer_data', {})

            c19 = {
                'name': data.get('name',{}),
                'c19_price': mkt_data.get('current_price', {}).get('usd', 0),
                'c19_mkt_cap': mkt_data.get('market_cap', {}).get('usd', 0),
                'c19_twitter_followers': com_data.get('twitter_followers', 0),
                'c19_reddit_post_48h': com_data.get('reddit_average_posts_48h', 0),
                'c19_reddit_avg_comments': com_data.get('reddit_average_comments_48h', 0),
                'c19_reddit_subs': com_data.get('reddit_subscribers', 0),
                'c19_reddit_accounts_active': com_data.get('reddit_accounts_active_48h', 0),
                'c19_forks': dev_data.get('forks', 0),
                'c19_stars':dev_data.get('stars', 0), 
                'c19_pull_merged': dev_data.get('pull_requests_merged', 0),
                'c19_pull_contributors': dev_data.get('pull_request_contributors', 0),
                }

            c19_data.append(c19)
            time.sleep(7) 
    
    df = pd.DataFrame(c19_data)
    df.to_csv(file_name, index = False)
    
    return df
    

In [19]:
def current_crypto_data(cryptos, file_name):

    # Create a DataFrame from the dictionary
    crypto_data = []

    for crypto in cryptos:
        # Define the API URL
        url = f'https://api.coingecko.com/api/v3/coins/{crypto}'

        # Make a GET request to the API
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()

            # Extract nested data
            developer_data = data.get("developer_data", {})
            community_data = data.get("community_data", {})
            market_data = data.get("market_data", {})
            links = data.get("links", {})

            # Create a dictionary with the desired information
            current_crypto_data = {
                "name": data.get("name", ""),
                "symbol": data.get("symbol", ""),
                "categories": ", ".join(data.get("categories", [])),
                "market_cap": market_data.get("market_cap", {}).get("usd", 0),
                "market_cap_ranking": data.get("market_cap_rank", 0),
                "current_price": market_data.get("current_price", {}).get("usd", 0),
                "homepage": links.get("homepage", [""])[0],
                "repos_url": links.get("repos_url", {}).get("github", ""),
                "sentiment_votes_up_percentage": data.get("sentiment_votes_up_percentage", 0),
                "sentiment_votes_down_percentage": data.get("sentiment_votes_down_percentage", 0),
                "watchlist_portfolio_users": data.get("watchlist_portfolio_users", 0),
                "coingecko_rank": data.get('coingecko_rank', 0),
                "coingecko_score": data.get("coingecko_score", 0),
                "developer_score": data.get("developer_score", 0),
                "community_score": data.get("community_score", 0),
                "liquidity_score": data.get("liquidity_score", 0),
                "public_interest_score": data.get("public_interest_score", 0),
                "ath_price": market_data.get("ath", {}).get("usd", 0),
                "ath_date": market_data.get("ath_date", {}).get("usd", ""),
                "atl_price": market_data.get("atl", {}).get("usd", 0),
                "atl_date": market_data.get("atl_date", {}).get("usd", ""),
                "price_change_24h": market_data.get("price_change_percentage_24h", 0),
                "price_change_7d": market_data.get("price_change_percentage_7d", 0),
                "price_change_30d": market_data.get("price_change_percentage_30d", 0),
                "price_change_60d": market_data.get("price_change_percentage_60d", 0),
                "price_change_1y": market_data.get("price_change_percentage_1y", 0),
                "total_supply": market_data.get("total_supply", 0),
                "max_supply": market_data.get("max_supply", 0),
                "circulating_supply": market_data.get("circulating_supply", 0),
                "last_updated": data.get("last_updated", ""),
                "twitter_followers": community_data.get("twitter_followers", 0),
                "telegram_channel_users": community_data.get("telegram_channel_user_count", 0),
                "forks": developer_data.get("forks", 0),
                "stars": developer_data.get("stars", 0),
                "pull_requests_merged": developer_data.get("pull_requests_merged", 0),
                "contributors": developer_data.get("pull_request_contributors", 0),
                "commit_count_4_weeks": developer_data.get("commit_count_4_weeks", 0)
            }


            crypto_data.append(current_crypto_data)
            time.sleep(5)
            
    df = pd.DataFrame(crypto_data)
    crypto_data.to_csv(file_name, index = False)
    
    return df